# Projeto 2 - Classificador Automático de Sentimento

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 19/Set até às 23:59.<br />
Grupo: 2 ou 3 pessoas - grupos com 3 pessoas terá uma rubrica diferenciada.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO gravar a key do professor no arquivo**


### Entrega Intermediária: Check 1 - APS 2

Até o dia 10/Set às 23:59, xlsx deve estar no Github com as seguintes evidências: 

  * Produto escolhido.
  * Arquivo Excel contendo a base de treinamento e a base de testes já classificadas.

Sugestão de leitura:<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

In [631]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk

___

## Parte I - Adquirindo a Base de Dados

Acessar o notebook **Projeto-2-Planilha** para realizar a coleta dos dados. O grupo deve classificar os dados coletados manualmente.

___
## Parte II - Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Não se esqueça de implementar o Laplace Smoothing (https://en.wikipedia.org/wiki/Laplace_smoothing).

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.

Escreva o seu código abaixo:

In [632]:
data = pd.read_excel("mcdonalds.xlsx",sheet_name = "Treinamento")
colunas = ["Treinamento", "Classificação"]
data = data.loc[:,colunas]


#Limpando caracteres irrelevantes:

def deletar_caracteres(coluna):
    coluna = coluna.replace("."," ").replace(":"," ").replace(","," ").replace("'","").replace('"', " ").replace\
    ("rt"," ").replace("#"," ").replace("-","")
    return coluna

    

data['Treinamento'] = data['Treinamento'].apply(deletar_caracteres)



In [633]:
#Dividindo entre pos, neg e irrelevante
pos = data[data["Classificação"] == 2]
neg = data[data["Classificação"] == 1]
irrel = data[data["Classificação"] == 0] 

    

In [634]:
dic = data["Treinamento"]
dic_palavras_pos = {}
dic_palavras_neg = {}
dic_palavras_irrel = {}

for i in pos["Treinamento"]:
    x = i.split()
    for e in x:
        if e not in dic_palavras_pos:
            dic_palavras_pos[e] = 1
        
        elif e in dic_palavras_pos:
            dic_palavras_pos[e] += 1

for i in neg["Treinamento"]:
    x = i.split()
    for e in x:
        if e not in dic_palavras_neg:
            dic_palavras_neg[e] = 1
        
        elif e in dic_palavras_neg:
            dic_palavras_neg[e] += 1

            
for i in irrel["Treinamento"]:
    x = i.split()
    for e in x:
        if e not in dic_palavras_irrel:
            dic_palavras_irrel[e] = 1
        
        elif e in dic_palavras_irrel:
            dic_palavras_irrel[e] += 1

            






In [635]:


df2 = pd.DataFrame.from_dict(dic_palavras_pos, orient='index')
df1 =  pd.DataFrame.from_dict(dic_palavras_neg,  orient='index')
df0 =  pd.DataFrame.from_dict(dic_palavras_irrel,  orient='index')


df2.columns = ["Positivo"]
df1.columns = ["Negativo"]
df0.columns = ["Irrelevante"]


Dataframe =  df2.join(df1, how='outer')
Dataframe = Dataframe.join(df0, how='outer')

Dataframe = Dataframe.fillna(0)
Dataframe.reset_index(level=0, inplace=True)
Dataframe.rename(columns = {"index":'Palavras'}, inplace=True)

Dataframe['Positivo'] = Dataframe['Positivo'].astype(int)
Dataframe['Negativo'] = Dataframe['Negativo'].astype(int)
Dataframe['Irrelevante'] = Dataframe['Irrelevante'].astype(int)







In [636]:
# Total de palavras positivas, negativas e irrelevantes

quantidade_pos = 0
quantidade_neg = 0
quantidade_irrel = 0  


for e in Dataframe.Positivo:
    quantidade_pos +=e
    
       
for k in Dataframe.Negativo:
        quantidade_neg += k       
        
for f in Dataframe.Irrelevante:
        quantidade_irrel +=f 
        
print(quantidade_pos)    
print(quantidade_neg)
print(quantidade_irrel)  







     
    


1635
893
2012


In [637]:
dic2 = {}
dic1 = {}
dic0 = {}
lista2 = []
lista1 = []
lista0 = []

for e in Dataframe.Positivo:
    lista2.append((e/quantidade_pos))
    
for e in Dataframe.Negativo:
    lista1.append((e/quantidade_pos))
    
for e in Dataframe.Irrelevante:
    lista0.append((e/quantidade_pos))
    
for i in range(0,len(Dataframe.Positivo)):
    dic2[Dataframe.Palavras[i]] = lista2[i]
    
for i in range(0,len(Dataframe.Negativo)):
    dic1[Dataframe.Palavras[i]] = lista1[i]
    
for i in range(0,len(Dataframe.Irrelevante)):
    dic0[Dataframe.Palavras[i]] = lista0[i]
        
index = pd.RangeIndex(start=0, stop=1, step=1)  

    
dc2 = pd.DataFrame(data=dic2,index=index)
x=dc2.T
dc1 = pd.DataFrame(data=dic1,index=index)
y=dc1.T
dc0 = pd.DataFrame(data=dic0,index=index)
z=dc0.T
x.columns = ["P/Positivo"]
y.columns = ["P/Negativo"]
z.columns = ["P/Irrelevante"]

Dataframe2 =  x.join(y, how='inner')
Dataframe2 = Dataframe2.join(z, how='inner')

In [642]:
total_palavras = quantidade_pos+ quantidade_neg+ quantidade_irrel

#Probabilidade de ser positivo = palavras positivas/total de palavras
Pp = (quantidade_pos/(total_palavras))
                     
#Probabilidade de ser negativo = palavras negativas/total de palavras                     
Pn = (quantidade_neg/(total_palavras))

#Probabilidade de ser negativo = palavras negativas/total de palavras                     
Pi = (quantidade_irrel/(total_palavras))

print(Pp)
print(Pn)
print(Pi)


# probabilidade da palavra/sentimento: 

Dataframe2


 
#probabilidade da palavra aparecer em qualquer sentimento:




0.3601321585903084
0.19669603524229076
0.4431718061674009


,P/Positivo,P/Negativo,P/Irrelevante
&amp;,0.000000,0.000000,0.000612
&gt;,0.000000,0.000000,0.004893
(@,0.001223,0.000000,0.000000
(sabado,0.000000,0.000612,0.000000
//t,0.014067,0.008563,0.039755
000,0.000612,0.000000,0.000000
01,0.000612,0.000000,0.000000
1,0.000000,0.000612,0.000000
10,0.000612,0.000000,0.000612
123m,0.000000,0.000000,0.000612


___
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Obrigatório para grupos de 3 alunos:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

___
## Concluindo

Escreva aqui a sua conclusão.<br /> 
Faça um comparativo qualitativo sobre as medidas obtidas.<br />
Explique como são tratadas as mensagens com dupla negação e sarcasmo.<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).
